In [0]:
from pyspark.sql.functions import *

In [0]:
credit_cards_table = "incremental_load.default.credit_cards"
cdc_df = spark.read.format("delta") \
    .option("readChangeData", "true") \
    .option("startingVersion", 0) \
    .table(credit_cards_table)
cdc_df.display()


In [0]:
new_records = cdc_df.filter(cdc_df["_change_type"].isin("update_postimage", "insert")) \
    .withColumn("valid_from", current_timestamp()) \
    .withColumn("valid_to", lit(None).cast("timestamp")) \
    .withColumn("is_current", lit(True)) \
    .drop("_change_type", "_commit_version")
new_records.display() 

In [0]:
from delta.tables import DeltaTable

In [0]:
scd2_catalog_table = "incremental_load.default.credit_cards_scd2" 
try:
    print("Table exists, updating table")
    scd2_table = DeltaTable.forName(spark, scd2_catalog_table)
except:
    print("Table does not exist, creating new table")
    new_records.write.format("delta").mode("overwrite").saveAsTable(scd2_catalog_table)
    scd2_table = DeltaTable.forName(spark, scd2_catalog_table)
    print("Table created")


scd2_table.alias("t").merge(
    new_records.alias("s"),
    "t.card_id = s.card_id AND t.is_current = true"
).whenMatchedUpdate(set={
    "valid_to": current_timestamp(),
    "is_current": lit(False)
}).execute()

scd2_table.alias("t").merge(
    new_records.alias("s"),
    "t.card_id = s.card_id AND t.is_current = false"
).whenNotMatchedInsertAll().execute()

spark.read.table(scd2_catalog_table).display()

